# Workflow 1, Module 2

One approach to solving this module is to not define quite so tightly what's going on at the subquestion level in terms of enrichements and archetypes and so on, but simply pass the question to ROBOKOP and let its scoring bring the the best answers to the top.  Here we will use the quick service to start with a disease, find relevant phenotypes, and from there find genetic conditions. The answers will come out ranked by path.

For more details, see the "quick" notebook in greengamma/general.

First, we'll have a quick function that calls the quick service, and some functions for properly creating the question. 

In [18]:
#Load some functions for calling the quick service and parsing its output

import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from gg_functions import parse_answer, quick, get_view_url

#To make nicer looking outputs
from IPython.core.display import display, HTML

## Without pathway expansion

The module as written starts with a genetic condition, gets genes, and then gets chemicals interacting with those genes.   On the presumption that this is insufficient to get into new chemical space, it also adds the expansion from genes to other genes via pathways.   But lets originally start without the pathway expansion.  So we'll just do 

`genetic_condition` to `gene` to `chemical_substance`

In [2]:
def no_expand_question(disease_id):
    q = {
        "machine_question": {
            "nodes": [
                {
                    "id": "n0",
                    "type": "disease",
                    "curie": disease_id
                },
                {
                    "id": "n1",
                    "type": "gene"
                },
                {
                    "id": "n2",
                    "type": "chemical_substance"
                }
            ],
            "edges": [
                {
                    "id": "e0",
                    "source_id": "n0",
                    "target_id": "n1"
                },
                {
                    "id": "e1",
                    "source_id": "n1",
                    "target_id": "n2"
                }
            ]
        }
    }
    return q
    

In [3]:
glucose_intolerance = 'MONDO:0001076' #glucose intolerance
sids = 'MONDO:0010086'

In [4]:
glucose_intolerance_question = no_expand_question(glucose_intolerance)
gi_answer = quick(glucose_intolerance_question)

Return Status: 200


In [5]:
gi_frame = parse_answer(gi_answer,node_list=['n2'],edge_list=[])
gi_frame

,n2 - id,n2 - name,score
0,CHEBI:17234,glucose,0.804310
1,CHEBI:17234,glucose,0.785865
2,CHEBI:17234,glucose,0.759348
3,CHEBI:17234,glucose,0.759009
4,CHEBI:17234,glucose,0.720393
5,CHEBI:17234,glucose,0.719615
6,CHEBI:17234,glucose,0.663424
7,CHEBI:16469,17beta-estradiol,0.445326
8,CHEBI:9516,thapsigargin,0.440844
9,CHEBI:50122,rosiglitazone,0.438941


In [6]:
SIDS_question = no_expand_question(sids)
SIDS_answer = quick(SIDS_question)
SIDS_frame = parse_answer(SIDS_answer,node_list=['n2'],edge_list=[])
SIDS_frame

Return Status: 200


,n2 - id,n2 - name,score
0,CHEBI:28593,quinidine,0.441819
1,CHEBI:27958,cocaine,0.439788
2,CHEBI:28487,reserpine,0.438438
3,CHEBI:6809,methamphetamine,0.435212
4,CHEBI:33216,bisphenol A,0.431663
5,CHEBI:7983,pentobarbital,0.430733
6,CHEBI:27958,cocaine,0.429535
7,CHEBI:28790,serotonin,0.428917
8,CHEBI:6809,methamphetamine,0.427845
9,CHEBI:5118,fluoxetine,0.427508


## With pathway expansion

We can also run the question as asked by opening up the genes to those involved in the same biological processes

In [7]:
def pathway_expand_question(disease_id):
    q = {
        "machine_question": {
            "nodes": [
                {
                    "id": "n0",
                    "type": "disease",
                    "curie": disease_id
                },
                {
                    "id": "n1",
                    "type": "gene"
                },
                {
                    "id": "n2",
                    "type": "biological_process_or_activity"
                },
                {
                    "id": "n3",
                    "type": "gene"
                },
                {
                    "id": "n4",
                    "type": "chemical_substance"
                }
            ],
            "edges": [
                {
                    "id": "e0",
                    "source_id": "n0",
                    "target_id": "n1"
                },
                {
                    "id": "e1",
                    "source_id": "n1",
                    "target_id": "n2"
                },
                {
                    "id": "e2",
                    "source_id": "n2",
                    "target_id": "n3"
                },
                {
                    "id": "e3",
                    "source_id": "n3",
                    "target_id": "n4"
                }
            ]
        }
    }
    return q

In [ ]:
gi_exp_question = pathway_expand_question(glucose_intolerance)
gi_exp_answer = quick(gi_exp_question,max_connectivity=500)
gi_exp_frame = parse_answer(gi_exp_answer,node_list=['n1','n2','n3','n4'],edge_list=[],node_properties=['name'])
view_url = get_view_url(gi_exp_answer)
display(HTML(f'<a href={view_url}>View Answer in ROBOKOP</a>'))
display(gi_exp_frame)

In [ ]:
sids_exp_question = pathway_expand_question(sids)
sids_exp_answer = quick(sids_exp_question,max_connectivity=500)
view_sids_url = get_view_url(sids_exp_answer)
display(HTML(f'<a href={view_sids_url}>View Answer in ROBOKOP</a>')
sids_exp_frame = parse_answer(sids_exp_answer,node_list=['n0','n1','n2','n3','n4'],edge_list=[],node_properties=['name'])
display(sids_exp_frame)